# Export RGB crops from channel overviews

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
from improc.io import parse_collection, DCAccessor
from improc.metadata import imagej_imsave
from skimage.io import imsave
from holoviews import opts
from inter_view.figures import CompositeFigureMaker
from inter_view.utils import make_composite
DCAccessor.register()

from inter_view.dashboards import CompositeExportDashBoard

import holoviews as hv
hv.extension('bokeh', width=100)

# config
configure file parsing and channel display options. Remove the `intensity_bounds` for autoscaling

In [ ]:
px_size = 0.322
basedir = '../../data/2D'
outdir = '../../data/2D_out'
data_pattern = '{platedir}/{subdir}/{f1}_{f2}_{f3}_{row}{col:02d}_T{T:04d}F{field:03d}L{L:02d}A{A:02d}Z{zslice:02d}C{channel:02d}.{ext}'
index = ['platedir','subdir','row', 'col', 'field', 'zslice', 'channel']

channel_config = {'marker1':{'cmap':'cyan','intensity_bounds':(0,2**16), 'slider_limits':(0,2**16)},
                  'marker2':{'cmap':'magenta','intensity_bounds':(0,2**16), 'slider_limits':(0,2**16)},
                  'marker3':{'cmap':'yellow','intensity_bounds':(0,2**16), 'slider_limits':(0,2**16)}}

ch_lut = {1: 'marker1', 2: 'marker2', 3: 'marker3'}


opts.defaults(opts.Image('channel', frame_width=1500))

# parse files

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)

# reindex with label instead of channel id
df.reset_index('channel', inplace=True)
df['ch_label'] = df.channel.map(ch_lut)
df.set_index('ch_label', append=True, inplace=True)

df = df.dc[:,'TIF_OVR_MIP']
df.reset_index(['field', 'zslice'], inplace=True)
df

# Interactive figure export dashboard

- To draw a roi, activate the "Box Edit Tool" on the right of the image, press shift or double click and drag the mouse.
- for fine cursor adjustment, click once on the cursor and use the arrow keys (tested in chrome)
- exporting a full overview (i.e. without selecting a ROI) might run out of memory

Export functions:
- export color tif: RGB tif with blended channels, same resolution as original
- export imagej: raw data saved as composite tif and cmap/intensity bounds metadata
- export figure: exports png rgb image with scale bar and legends

In [ ]:
from skimage.io import imread

def read_first_slice(path):
    '''Reads only the first slice and caches the last set of channels'''    
    return imread(path, img_num=0)

# wrap export functions
def export_imagej(path, imgs, cmaps, intensity_bounds, labels):
    img = np.stack(imgs, axis=-1)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    imagej_imsave(path, np.stack(imgs, axis=-1), cmaps, intensity_bounds)
export_imagej.name = 'export imagej'

def export_color_tif(path, imgs, cmaps, intensity_bounds, labels):
    pre, ext = os.path.splitext(path)
    path = pre + '_blended' + ext
    
    imgs = [rescale_intensity(img, in_range=b, out_range=np.uint8).astype(np.uint8) for img,b in zip(imgs, intensity_bounds)]
    rgb_img = make_composite(imgs, cmaps)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    imsave(path, rgb_img, compress=9)
export_color_tif.name = 'export color tif'
    
def export_figure(path, imgs, cmaps, intensity_bounds, labels):
    pre, ext = os.path.splitext(path)
    path = pre + '.png'
    
    cfm = CompositeFigureMaker(scale=px_size, show_legend=True, bar_lengths=[20,50], legend_pos='lower left', scale_bar_pos='lower right')
    fig = cfm(imgs, cmaps, intensity_bounds, labels)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    fig.savefig(path, bbox_inches='tight', pad_inches=0)
export_figure.name = 'export figure'

db = CompositeExportDashBoard(df=df.dc[:,['TIF_OVR_MIP'],:],
                              multi_select_levels=['ch_label'],
                              loading_fun=read_first_slice,
                              channel_config=channel_config,
                              export_funs=[export_color_tif, export_imagej, export_figure],
                              out_folder=outdir)

db.panel()